In [3]:
import gym
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import matplotlib.pyplot as plt  
import numpy as np
import random
from collections import deque, namedtuple


In [67]:
class Actor(nn.Module):
    
    def __init__(self, input_dim, output_dim, hidden_nodes):
        super(Policy, self).__init__()
        self.input_dim = input_dim
        self.output_dimput_dim = output_dim
        self.hn = hidden_nodes
        
        self.fc1 = nn.Linear(self.input_dim, self.hn)
        self.fc2 = nn.Linear(self.hn, self.hn)
        self.fc3 = nn.Linear(self.hn, self.output_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x, dim=1)) # This computes a distribution from which we can sample and take log of for the gradient decent
        return x 
    
class Critic(nn.Module):
    
    def __init__(self, input_dim, hidden_nodes):
        self.input_dim = input_dim
        self.hn = hidden_nodes
        
        self.fc1 = nn.Linear(self.input_dim, self.hn)
        self.fc2 = nn.Linear(self.hn , self.hn)
        self.fc3 = nn.Linear(self.hn, 1) # Outputs a Q value of the state
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

In [ ]:
def train(env, epochs, actor, critic):
    
    actor_optimizer = optim.Adam(actor.parameters())
    critic_optimizer = optim.Adam(critic.parameters())
    translate = namedtuple('translate',
                        ('state', 'action_prob', 'critic_q', 'reward', 'next_state', 'done'))

    

    for i in range(epochs):
        mini_batch = []
        obs = env.reset()
        
        while not done:
            obs = torch.from_numpy(obs).type(torch.float)
            # Do we need the exploration exploitation?
            action_prob  = actor(obs)
            m = Categorical(action_prob)
            action = action_prob.sample(m)
            action_prob = m.log_prob(action)
            
            next_obs, reward, done, info = env.step(action)
            critic_q = critic(next_obs)

            obs = torch.tensor(obs, dtype=torch.float32)
            reward = torch.tensor(reward, dtype=torch.float32)
            next_obs = torch.tensor(next_obs, dtype=torch.float32)

            mini_batch.append(translate(obs, action_prob, critic_q, reward, next_obs))
            
            obs = next_obs
            
        state = torch.stack(mini_batch.state)
        action_prob = torch.stack(mini_batch.action_prob)
        critic_q = torch.stack(mini_batch.critic_q)
        reward = torch.stack(mini_batch.reward)
        next_state = torch.stack(mini_batch.next_state)
        done = torch.stack(mini_batch.done)

        


    
    

In [5]:
env = gym.make("Acrobot-v1")
print(env.action_space)

Discrete(3)
